In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from qkeras import *
from keras.models import Model
from keras.layers import *
from telescope import *



2023-07-31 04:48:14.303022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 04:48:23.504950: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /uscms_data/d3/nswood/mambaforge/lib/:/uscms_data/d3/nswood/mambaforge/lib/python3.9/site-packages/nvidia/cudnn/lib:/uscms_data/d3/nswood/mambaforge/lib/:/uscms_data/d3/nswood/mambaforge/lib/python3.9/site-packages/nvidia/cudnn/lib:/uscms_data/d3/nswood/mambaforge/lib/:/uscms_data/d3/nswood/mambaforge/lib/python3.9/site-packages/nvidia/cudnn/lib:
2023-07-31 04:48:23.507017: W tensorflow/com

In [94]:
a = tf.random.uniform([2,96])

In [181]:
tf.expand_dims(indices, 1).shape

TensorShape([96, 1])

In [169]:
tensor = [[1, 1], [1, 1], [1, 1]]    # tf.rank(tensor) == 2
indices = [[0, 1], [2, 0]]           # num_updates == 2, index_depth == 2
updates = [5, 10]                    # num_updates == 2
print(tf.tensor_scatter_nd_update(tensor, indices, updates))


tf.Tensor(
[[ 1  5]
 [ 1  1]
 [10  1]], shape=(3, 2), dtype=int32)


In [167]:
tensor = [0, 0, 0, 0, 0, 0, 0, 0]    # tf.rank(tensor) == 1
indices = [[1], [3], [4], [7]]       # num_updates == 4, index_depth == 1
updates = [9, 10, 11, 12]            # num_updates == 4
print(tf.tensor_scatter_nd_update(tensor, indices, updates))


tf.Tensor([ 0  9  0 10 11  0  0 12], shape=(8,), dtype=int32)


In [147]:
 tf.expand_dims(tf.repeat( tf.expand_dims(indices, 0),2,axis=0),axis=2)

<tf.Tensor: shape=(2, 96, 1), dtype=int32, numpy=
array([[[  0],
        [  1],
        [  2],
        [  3],
        [  4],
        [  5],
        [  6],
        [  7],
        [  8],
        [  9],
        [ 12],
        [ 13],
        [ 16],
        [ 17],
        [ 18],
        [ 19],
        [ 20],
        [ 21],
        [ 22],
        [ 23],
        [ 24],
        [ 25],
        [ 28],
        [ 29],
        [ 32],
        [ 33],
        [ 34],
        [ 35],
        [ 36],
        [ 37],
        [ 38],
        [ 39],
        [ 40],
        [ 41],
        [ 44],
        [ 45],
        [ 48],
        [ 49],
        [ 50],
        [ 51],
        [ 52],
        [ 53],
        [ 54],
        [ 55],
        [ 56],
        [ 57],
        [ 60],
        [ 61],
        [ 64],
        [ 65],
        [ 66],
        [ 67],
        [ 68],
        [ 69],
        [ 70],
        [ 71],
        [ 72],
        [ 73],
        [ 76],
        [ 77],
        [ 80],
        [ 81],
        [ 82],
     

In [172]:
import tensorflow as tf
import numpy as np

a = np.random.rand(1,96)
# Convert 'numbers_list' to a TensorFlow constant
indices = tf.constant(numbers_list, dtype=tf.int32)

# Convert 'a' to a TensorFlow constant and reshape it to match the indices shape

# Update the values in 'result_array' at the specified indices with values from 'a_tf'
result_array = tf.tensor_scatter_nd_update(result_array, tf.expand_dims(indices, 1), a)

# Reshape the 'result_array' into an 8x4x4 tensor
result_array = tf.reshape(result_array, (8, 4, 4))

print(result_array.numpy())


InvalidArgumentError: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} Outer dimensions of indices and update must match. Indices shape: [96,1], updates shape:[1,96] [Op:TensorScatterUpdate]

In [156]:
import tensorflow as tf
import numpy as np

a = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 130, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 280, 290, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 440, 450, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 600, 610, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 760, 770, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 920, 930, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1080, 1090, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1240, 1250])

# Convert 'numbers_list' to a TensorFlow constant
indices = tf.constant(numbers_list, dtype=tf.int32)

# Convert 'a' to a TensorFlow constant and reshape it to match the indices shape
result_array = tf.zeros_like(a)
# Update the values in 'result_array' at the specified indices with values from 'a_tf'
result_array = tf.tensor_scatter_nd_update(result_array, tf.expand_dims(indices, 1), a)

# Reshape the 'result_array' into an 8x4x4 tensor
result_array = tf.reshape(result_array, (8, 4, 4))

print(result_array.numpy())


2023-07-31 06:38:27.005168: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at scatter_nd_op.cc:216 : INVALID_ARGUMENT: indices[72] = [96] does not index into shape [96]


InvalidArgumentError: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} indices[72] = [96] does not index into shape [96] [Op:TensorScatterUpdate]

In [55]:
numbers_list = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
    25, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 48, 49, 50, 51, 
    52, 53, 54, 55, 56, 57, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 
    77, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 96, 97, 98, 99, 100, 101, 
    102, 103, 104, 105, 108, 109, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 124, 125
]

print(numbers_list)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 77, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 108, 109, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 124, 125]


In [3]:
def mean_mse_loss(y_true, y_pred):
    # Calculate the squared difference between predicted and target values
    squared_diff = tf.square(y_pred - y_true)

    # Calculate the MSE per row (reduce_mean along axis=1)
    mse_per_row = tf.reduce_mean(squared_diff, axis=1)

    # Take the mean of the MSE values to get the overall MSE loss
    mean_mse_loss = tf.reduce_mean(mse_per_row)

    return mean_mse_loss

In [182]:
batch = 2000

#Specs
n_kernels = 8
n_encoded=16
conv_weightBits  = 6 
conv_biasBits  = 6 
dense_weightBits  = 6 
dense_biasBits  = 6 
encodedBits = 9
CNN_kernel_size = 3





input_enc = Input(batch_shape=(batch,9,9, 1))
sum_input = Input(batch_shape=(batch,1))
eta = Input(batch_shape =(batch,1))
x = QConv2D(n_kernels,
            CNN_kernel_size, 
#             padding='same',
            strides=2,
            kernel_quantizer=quantized_bits(bits=conv_weightBits,integer=0,keep_negative=1,alpha=1),
            bias_quantizer=quantized_bits(bits=conv_biasBits,integer=0,keep_negative=1,alpha=1),
            name="conv2d")(input_enc)
x = QActivation("quantized_relu(bits=8,integer=1)", name="act")(x)
x = Flatten()(x)
# x = Concatenate(axis=1)([x[:,:80],x[:,96:112]]) 
x = QDense(n_encoded, 
           kernel_quantizer=quantized_bits(bits=dense_weightBits,integer=0,keep_negative=1,alpha=1),
           bias_quantizer=quantized_bits(bits=dense_biasBits,integer=0,keep_negative=1,alpha=1),
           name="dense")(x)

x = concatenate([x,sum_input,eta],axis=1)
latent = x

input_dec = Input(batch_shape=(batch,18))
y = Dense(128)(input_dec)
y = ReLU()(y)
y = Reshape((4, 4, 8))(y)
y = Conv2DTranspose(1, (3, 3), strides=(2, 2))(y)
y = ReLU()(y)
recon = y



from tensorflow.keras.optimizers import Adam

encoder = keras.Model([input_enc,sum_input,eta], latent, name="encoder")
decoder = keras.Model([input_dec], recon, name="decoder")

cae = Model(
    inputs=[input_enc,sum_input,eta],
    outputs=decoder([encoder([input_enc,sum_input,eta])]),
    name="cae"
)

cae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5), loss=mean_mse_loss)
cae.summary()


(2000, 9, 9, 1)
Model: "cae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_52 (InputLayer)          [(2000, 9, 9, 1)]    0           []                               
                                                                                                  
 input_53 (InputLayer)          [(2000, 1)]          0           []                               
                                                                                                  
 input_54 (InputLayer)          [(2000, 1)]          0           []                               
                                                                                                  
 encoder (Functional)           (2000, 18)           2144        ['input_52[0][0]',               
                                                                  'input_53[0][0

In [25]:
def load_data(nfiles,batchsize):
    data_list = []

    for i in range(nfiles):
        if i == 0:
            dt = pd.read_csv('../ECON_AE_Development/AE_Data/1.csv').values
        else:
            dt_i = pd.read_csv(f'../ECON_AE_Development/AE_Data/{i+1}.csv').values
            dt = np.vstack([dt, dt_i])

        data_list.append(dt)

    data_tensor = tf.convert_to_tensor(np.concatenate(data_list), dtype=tf.float32)

    train_size = int(0.8 * len(data_tensor))
    test_size = len(data_tensor) - train_size

    # Split the data into training and test sets
    train_data, test_data = tf.split(data_tensor, [train_size, test_size], axis=0)

    # Extract specific tensors
    train_wafers = expand_tensor(train_data[:, 0:48])
    train_sum_calcq = tf.expand_dims(tf.reduce_sum(train_data[:, 0:48], axis=1), axis=1)
    train_eta = tf.expand_dims(train_data[:, -2], axis=1)

    test_wafers = expand_tensor(test_data[:, 0:48])
    test_sum_calcq = tf.expand_dims(tf.reduce_sum(test_data[:, 0:48], axis=1), axis=1)
    test_eta = tf.expand_dims(test_data[:, -2], axis=1)

    # Create data loaders for training and test data
    train_dataset = tf.data.Dataset.from_tensor_slices((train_wafers, train_sum_calcq, train_eta))
    train_loader = train_dataset.batch(batchsize).shuffle(buffer_size=train_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_wafers, test_sum_calcq, test_eta))
    test_loader = test_dataset.batch(batchsize).shuffle(buffer_size=test_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_loader, test_loader


In [26]:
def expand_tensor(input_tensor):
    arrange = np.array([28,29,30,31,0,4,8,12,
                         24,25,26,27,1,5,9,13,
                         20,21,22,23,2,6,10,14,
                         16,17,18,19,3,7,11,15,
                         47,43,39,35,35,34,33,32,
                         46,42,38,34,39,38,37,36,
                         45,41,37,33,43,42,41,40,
                             44,40,36,32,47,46,45,44])
    arrMask = np.array([1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,])
   
    inputdata = tf.reshape(tf.gather(input_tensor, arrange,axis =1), (input_tensor.shape[0],8, 8, 1))
#     inputdata *= tf.cast(tf.reshape(arrMask, (1, 8, 8)), dtype=inputdata.dtype)

    paddings = [(0, 0), (0, 1), (0, 1), (0, 0)]
    padded_tensor = tf.pad(inputdata, paddings, mode='CONSTANT', constant_values=0)

    return padded_tensor


In [18]:
len(train_loader)*batch

2764000

In [11]:
train_loader, test_loader = load_data(1,batch)

In [183]:
nepochs = 100
for epoch in range(nepochs):
    total_loss = 0
    for wafers, sum_calcq, eta in train_loader:
        # Train the CAE model on each batch
        tensor2_expanded = tf.expand_dims(tf.expand_dims(sum_calcq, -1), -1)

        if wafers.shape[0] != batch:
            break
        loss = cae.train_on_batch([wafers, sum_calcq, eta], wafers)
        total_loss += loss
        
    print(f"Epoch {epoch+1}/{nepochs}, Loss: {total_loss/(len(train_loader)*batch)}")


    

2023-07-31 06:55:12.750201: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/cae/encoder/act/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 1/100, Loss: 0.16153756910122596
Epoch 2/100, Loss: 0.03399047360233909
Epoch 3/100, Loss: 0.3102665162390117
Epoch 4/100, Loss: 0.31919509405613633
Epoch 5/100, Loss: 0.10831190894136553
Epoch 6/100, Loss: 0.215050784333915
Epoch 7/100, Loss: 0.4186238573180611
Epoch 8/100, Loss: 0.38946575976936243
Epoch 9/100, Loss: 0.21934110993276285
Epoch 10/100, Loss: 0.2045010686162239
Epoch 11/100, Loss: 0.5185253537939875
Epoch 12/100, Loss: 0.33309028568488647
Epoch 13/100, Loss: 0.20360406418509144
Epoch 14/100, Loss: 0.40674306669663074
Epoch 15/100, Loss: 0.08346010239182954
Epoch 16/100, Loss: 0.39887865645978626
Epoch 17/100, Loss: 0.0897992967172228
Epoch 18/100, Loss: 0.2778018697429497
Epoch 19/100, Loss: 0.16108096089549417
Epoch 20/100, Loss: 0.4275206747910738
Epoch 21/100, Loss: 0.3976863715196656
Epoch 22/100, Loss: 0.3710293976470461
Epoch 23/100, Loss: 0.190703836228499
Epoch 24/100, Loss: 0.32062857378616
Epoch 25/100, Loss: 0.10131027782015103
Epoch 26/100, Loss: 0.308

In [ ]:
def load_data(args):
    data_list = []

    for i in range(args.nfiles):
        if i == 0:
            dt = pd.read_csv('../ECON_AE_Development/AE_Data/1.csv').values
        else:
            dt_i = pd.read_csv(f'../ECON_AE_Development/AE_Data/{i+1}.csv').values
            dt = np.vstack([dt, dt_i])

        data_list.append(dt)

    data_tensor = tf.convert_to_tensor(np.concatenate(data_list), dtype=tf.float32)

    train_size = int(0.8 * len(data_tensor))
    test_size = len(data_tensor) - train_size

    # Split the data into training and test sets
    train_data, test_data = tf.split(data_tensor, [train_size, test_size], axis=0)

    # Extract specific tensors
    train_wafers = train_data[:, 0:48]
    train_sum_calcq = tf.reduce_sum(train_data[:, 0:48], axis=1)
    train_eta = train_data[:, -2]

    test_wafers = test_data[:, 0:48]
    test_sum_calcq = tf.reduce_sum(test_data[:, 0:48], axis=1)
    test_eta = test_data[:, -2]

    # Create data loaders for training and test data
    train_dataset = tf.data.Dataset.from_tensor_slices((train_wafers, train_sum_calcq, train_eta))
    train_loader = train_dataset.batch(args.batchsize).shuffle(buffer_size=train_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_wafers, test_sum_calcq, test_eta))
    test_loader = test_dataset.batch(args.batchsize).shuffle(buffer_size=test_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_loader, test_loader
